# Хроматическое число R^4

### Описание алгоритма

Алгоритм:

- строим решетку (grid) и соответствующее ей разбиение Вороного (vor4)
- Ищем все многогранники центрального региона и его диаметр
- составляем список граней центрального многогранника (list_faces)
- перебираю различные подрешетки для каждой ищу приведенный базис, нахожу длину минимального вектора и минимальное расстояние между векторами и их комбинациями:
- Ищем точку s (середина расстояния между точкой (0, 0, 0, 0) и центром ближайшего многогранника
- Нахожу расстояние от точки s до центрального многоранника
- Умножаю это расстояние на 2 и получаю расстояние между многранниками (центральным и ближайшим к нему многогранником из подрешетки)
- составляем словари, ключами которых являются координаты точек s, а значениями - минимальное хроматическое число (dict_det), максимальное запрещенное расстоянием (dict_dist), список матриц (dict_s), соответствующих данной точке s, хроматическому числу и запрещенному расстоянию
- составляем датафрейм (df), обьединяя все эти словари
- составляем список матриц для каждого подходящего хроматического числа (если запрещенное расстояние больше 1)

- central - список вершин центрального многогранника
- coords4 - список координат центров многогранников Вороного

- vor4.vertices - список координат вершин разбиенния Вороного
- vor4.regions - список индексов координат вершин многогранников разбиения Вороного (vor4.vertices)
- vor4.ridge_vertices - координаты вершин 3-мерных граней разбиенния Вороного через их индексы в vor4.vertices
- vor4.ridge_points - список пар центров многогранников Вороного между которыми есть ребро 2х многогранников через индексы в coords4 (по сути - соседние многогранники)

- edge_central_coords - список списков вершин каждой 3-мерной грани центрального многогранника
- edge_central - список индексов (в vor4.vertices)  координат каждой 3-мерной грани 
- list_faces - список 3х мерных граней центрального многогранника с разбивкой каждой 3х мерной грани на 2-мерные
- list_pairs - список центров многогранников Вороного, соседних с центральным (индексы в coords4)

- list_neigh_points - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника)
- dict_point_edge - словарь: ключ - координаты соседней точки, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
- index_point_edge - индекс грани в списке граней центрального многогранника (edge_central_coords)
- list_ridge_edge - список граней, в том же порядке, что и соседние точки (list_neigh_points) или что и соответствующие вектора нормали к данным граням
- v_norm - список нормированных векторов нормали (в том же порядке, что и изначальные точки(вектора) нормали)
- point_face_norm - точки, принадлежащие грани и лежащие на векторе нормали (середина вектора, соединяющего центры)

### Построение многогранника Вороного

In [ ]:
import copy
from itertools import *
from copy import deepcopy

from Polyhedra import *

grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

vor4 = VoronoiPolyhedra(grid)
vor4.build()

In [ ]:
# вершины диаграммы Вороного
vor4.vertices

In [ ]:
# регионы

vor4.regions[1]

In [ ]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for i in vor4.regions:
    if -1 not in i:
        vor4.regions_finite.append(i)
        count += 1
        
print(count)

In [ ]:
# максимальный размер региона (по числу вершин)

len_r_max = 0 # размер максимального региона
count_r = 0 # количество таких регионов

for i in range(len(vor4.regions)):
    len_r = len(vor4.regions[i])
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print('количество вершин =', len_r_max, 'количество =', count_r)

In [ ]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник central

sum_dist_min = 1000

for i in range(1, len(vor4.regions)):
    len_r = len(vor4.regions[i])

    if -1 in vor4.regions[i]: continue # регион неограничен
        
    l = 0
    
    # для i региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
    for j in range(len_r):
        l += distance.euclidean(vor4.vertices[vor4.regions[i][j]], [0.0, 0.0, 0.0, 0.0])

    if l < sum_dist_min:
        sum_dist_min = l
        v_min = i

        
print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

central = vor4.vertices[vor4.regions[v_min]]

In [ ]:
# индексы вершин центрального региона (координаты vor4.vertices)

vor4.regions[v_min]

In [ ]:
# проверка на ортогональность

for i in range(len(list_ridge_edge[10]) - 1):
    for j in range (1, len(list_ridge_edge[10])):
        print(np.array(list_neigh_points[10]).dot(np.array(list_ridge_edge[10][i] - list_ridge_edge[10][j])))
         

In [ ]:
def gram_schmidt(v):

    w_1 = copy.deepcopy(v[0])
    w_n = copy.deepcopy(w_1)
    w_array = [deepcopy(w_n)]
    
    for n in range(1, len(v)):
        v_n = copy.deepcopy(v[n])
        w_n = copy.deepcopy(v_n)

        for j in range(n):
            w_j = deepcopy(w_array[j])
            if not any(w_j):
                continue
            w_n -= np.dot(v_n, w_j) / np.dot(w_j, w_j) * w_j
        
        w_array += [w_n]
          
    return w_array

In [ ]:
d = 0.75

def mu(b_i, b_j):
    return np.dot(b_i, b_j) / np.dot(b_j, b_j) if np.dot(b_j, b_j) != 0 else 0


def LLL(l_basis):

    ortho = gram_schmidt(l_basis)

    k = 1
    n = len(ortho)

    while k < n:
        for j in range(k - 1, -1, -1):
            proj = mu(l_basis[k], ortho[j])
            if abs(proj) > 1/2:
                l_basis[k] -= l_basis[j] * round(proj)
                ortho = copy.deepcopy(gram_schmidt(l_basis))
            #print(l_basis, 111)

        if np.dot(ortho[k], ortho[k]) >= (d - (mu(l_basis[k], ortho[k-1]))**2) * (np.dot(ortho[k-1], ortho[k-1])):
            #print(l_basis, 222)
            k += 1

        else:
            s = copy.deepcopy(l_basis[k-1])
            l_basis[k-1] = copy.deepcopy(l_basis[k])
            l_basis[k] = copy.deepcopy(s)
            ortho = gram_schmidt(l_basis)
            k = max(k-1, 1)
            #print(l_basis, 333)

    return l_basis

In [ ]:
def pad_lists_with_ones(list_of_lists):
    # Проходим по каждому списку в основном списке
    for i in range(len(list_of_lists)):
        # Если длина списка меньше 4
        if len(list_of_lists[i]) < 4:
            # Добавляем в начало столько единиц, сколько не хватает до 4
            list_of_lists[i] = [1] * (4 - len(list_of_lists[i])) + list_of_lists[i]
    return list_of_lists
